## Data Preparation

In [13]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import mlflow
import mlflow.sklearn
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Set plot style for better visuals
sns.set(style="whitegrid")


In [14]:
# Load the datasets
fraud_data = pd.read_csv('../data/cleaned_data_1.csv')
credit_data = pd.read_csv('../data/cleaned_data_2.csv')

In [ ]:
# Prepare data for the model (e-commerce)
X1 = fraud_data.drop(columns=['class'])  
X2 = credit_data.drop(columns=['Class'])  

y1 = fraud_data['class']  
y2 = credit_data['Class']  


print(np.unique(y1, return_counts=True))
print(np.unique(y2, return_counts=True))

(array([0, 1]), array([125849,  12997]))
(array([0, 1]), array([283253,    473]))


In [17]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Check for non-numeric columns in X_train1
non_numeric_columns = X_train1.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_columns)

# Convert categorical columns to numeric (if any)
for column in non_numeric_columns:
    # If it's a binary categorical column (like 'sex'), use LabelEncoder
    if len(X_train1[column].unique()) <= 2:
        label_encoder = LabelEncoder()
        X_train1[column] = label_encoder.fit_transform(X_train1[column])
        X_test1[column] = label_encoder.transform(X_test1[column])
    else:
        # For columns with more than two categories, use OneHotEncoder
        X_train1 = pd.get_dummies(X_train1, columns=[column], drop_first=True)
        X_test1 = pd.get_dummies(X_test1, columns=[column], drop_first=True)

# Now fit the Logistic Regression model
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train1, y_train1)
y_pred_log_reg = log_reg.predict(X_test1)

# Print classification report
print("Logistic Regression (fraud data):")
print(classification_report(y_test1, y_pred_log_reg))


In [27]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train1, y_train1)
y_pred_log_reg = log_reg.predict(X_test1)
print("Logistic Regression (fraud data):")
print(classification_report(y_test1, y_pred_log_reg))

Logistic Regression (fraud data):
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     25193
           1       0.00      0.00      0.00      2577

    accuracy                           0.91     27770
   macro avg       0.45      0.50      0.48     27770
weighted avg       0.82      0.91      0.86     27770



In [30]:
# Decision Tree Classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train1, y_train1)
y_pred_dt = dt_clf.predict(X_test1)
print("Decision Tree (fraud data):")
print(classification_report(y_test1, y_pred_dt))

Decision Tree (fraud data):
              precision    recall  f1-score   support

           0       0.96      0.94      0.95     25193
           1       0.50      0.58      0.54      2577

    accuracy                           0.91     27770
   macro avg       0.73      0.76      0.74     27770
weighted avg       0.91      0.91      0.91     27770



In [31]:
# Random Forest Classifier (already implemented)
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train1, y_train1)
y_pred_rf = rf_clf.predict(X_test1)
print("Random Forest (fraud data):")
print(classification_report(y_test1, y_pred_rf))

Random Forest (fraud data):
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     25193
           1       1.00      0.55      0.71      2577

    accuracy                           0.96     27770
   macro avg       0.98      0.77      0.84     27770
weighted avg       0.96      0.96      0.95     27770



In [32]:
# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(n_estimators=50, learning_rate=0.1, max_depth=3, random_state=42)
gb_clf.fit(X_train1, y_train1)
y_pred_gb = gb_clf.predict(X_test1)
print("Gradient Boosting (fraud data):")
print(classification_report(y_test1, y_pred_gb))

Gradient Boosting (fraud data):
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     25193
           1       1.00      0.55      0.71      2577

    accuracy                           0.96     27770
   macro avg       0.98      0.77      0.84     27770
weighted avg       0.96      0.96      0.95     27770



In [34]:
# Multi-Layer Perceptron (MLP)
mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
mlp_clf.fit(X_train1, y_train1)
y_pred_mlp = mlp_clf.predict(X_test1)
print("MLP Classifier (fraud data):")
print(classification_report(y_test1, y_pred_mlp))

MLP Classifier (fraud data):
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     25193
           1       0.10      0.00      0.00      2577

    accuracy                           0.91     27770
   macro avg       0.50      0.50      0.48     27770
weighted avg       0.83      0.91      0.86     27770



In [ ]:
# RNN Model
from keras.layers import SimpleRNN
rnn_model = Sequential()
rnn_model.add(SimpleRNN(64, activation='relu', input_shape=(X_train1.shape[1], 1)))
rnn_model.add(Dense(1, activation='sigmoid'))
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
rnn_model.fit(X_train1.values.reshape(-1, X_train1.shape[1], 1), y_train1, epochs=10, batch_size=32)
y_pred_rnn = (rnn_model.predict(X_test1.values.reshape(-1, X_test1.shape[1], 1)) > 0.5).astype(int)
print("RNN (fraud data):")
print(classification_report(y_test1, y_pred_rnn))

..............................................

## Experiments

In [33]:
datasets = {
    "fraud_data": (X_train1, y_train1, X_test1, y_test1),
    "credit_data": (X_train2, y_train2, X_test2, y_test2),
}

In [2]:
# Defining each model
models = [
    # Logistic Regression
    (
        "Logistic Regression", 
        LogisticRegression(C=1, solver='liblinear')
    ),
    
    # Random Forest
    (
        "Random Forest", 
        RandomForestClassifier(n_estimators=30, max_depth=3)
    ),

    # Decision Tree
    (
        "Decision Tree", 
        DecisionTreeClassifier(max_depth=5)
    ),
    
    # Multi-Layer Perceptron (MLP)
    (
        "MLP Classifier", 
        MLPClassifier(hidden_layer_sizes=(100,), max_iter=300)
    ),
    
    # # Convolutional Neural Network (CNN)
    # (
    #     "CNN", 
    #     Sequential([
    #         Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    #         Flatten(),
    #         Dense(128, activation='relu'),
    #         Dense(1 , activation='sigmoid')
    #     ])
    # ),
    
    # # Recurrent Neural Network (RNN)
    # (
    #     "RNN", 
    #     Sequential([
    #         SimpleRNN(50, input_shape=(100, 1), activation='relu'),
    #         Dense(1, activation='sigmoid')
    #     ])
    # ),
    
    # # Long Short-Term Memory (LSTM)
    # (
    #     "LSTM", 
    #     Sequential([
    #         LSTM(50, input_shape=(100, 1), activation='relu'),
    #         Dense(1, activation='sigmoid')
    #     ])
    # )
]


NameError: name 'LogisticRegression' is not defined